# Miniproject 1

In [ ]:
!pip install pycosat
!pip install sympy


## Imports and Utilities
**Note**: these imports and functions are available in catsoop. You do not need to copy them in.

In [ ]:
import numpy as np
from sympy import Symbol, And, Or, satisfiable



## Problems

### Warmup 1
Use sympy to determine whether the following formula is satisfiable:
$(\neg x_1 \land x_2) \Rightarrow ((x_2 \lor x_3) \land (x_1 \lor \neg x_3))$.


For reference, our solution is **3** lines of code.

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 

def formula1_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  a,b,c = symbols("a b c")
  return satisfiable((~a | b) & (b | c) & (a | ~c)) != False

Tests

In [ ]:

assert formula1_is_satisfiable() == True
print('Tests passed.')

Tests passed.


### Warmup 2
Use sympy to determine whether the following formula is satisfiable:
$(x_1 \lor x_2) \land (\neg x_1 \lor \neg x_2) \land (x_1 \lor \neg x_2) \land (\neg x_1 \lor x_2)$.


For reference, our solution is **3** lines of code.

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 

def formula2_is_satisfiable():
  """Determines whether the above formula is satisfiable.

  Returns:
    is_satisfiable: A bool indicating whether the formula is satisfiable.
  """
  a,b = symbols("a b")
  return satisfiable((a | b) & (~a | b) & (a | ~b) & (~a | ~b)) != False

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 


a,b,c,d = symbols("a b c d")
a = false
if not a:
  print(satisfiable((False&b) | (c&d)))

{d: True, c: True}


Tests

In [ ]:

assert formula2_is_satisfiable() == False
print('Tests passed.')

Tests passed.


### Warmup 3
Prove to yourself that the SAT solver is doing something more clever than enumerating truth tables. Write a formula involving 100 variables for which the SAT solver is able to quickly find a solution. (Hint: assume that the SAT solver is like DPLL. What kinds of formulas would be especially easy for DPLL to satisfy?)

For reference, our solution is **1** lines of code.

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 

def create_large_solvable_formula():
  """Return a sympy formula with at least 100 variables that `satisfiable` can
  quickly solve.

  Returns:
    formula: A sympy logical formula.
  """
  vars = []
  for i in range(100):
    vars.append(symbols(f'x{i}'))

  formula = vars[0]
  for i in range(1, 100):
    formula = formula | vars[i]

  print(formula)
  return formula



Tests

In [ ]:
large_formula = create_large_solvable_formula()
assert len(large_formula.free_symbols) >= 100
result = satisfiable(large_formula)
print('Tests passed.')

x0 | x1 | x10 | x11 | x12 | x13 | x14 | x15 | x16 | x17 | x18 | x19 | x2 | x20 | x21 | x22 | x23 | x24 | x25 | x26 | x27 | x28 | x29 | x3 | x30 | x31 | x32 | x33 | x34 | x35 | x36 | x37 | x38 | x39 | x4 | x40 | x41 | x42 | x43 | x44 | x45 | x46 | x47 | x48 | x49 | x5 | x50 | x51 | x52 | x53 | x54 | x55 | x56 | x57 | x58 | x59 | x6 | x60 | x61 | x62 | x63 | x64 | x65 | x66 | x67 | x68 | x69 | x7 | x70 | x71 | x72 | x73 | x74 | x75 | x76 | x77 | x78 | x79 | x8 | x80 | x81 | x82 | x83 | x84 | x85 | x86 | x87 | x88 | x89 | x9 | x90 | x91 | x92 | x93 | x94 | x95 | x96 | x97 | x98 | x99
Tests passed.


### Search and Rescue Inference
Write a program that takes a grid as input and infers unknown values.

Your program should output a new grid with all determinable unknown values replaced with the inferred value. If an unknown value cannot be determined, it should be left unknown.

**Your program should use sympy.**


For reference, our solution is **53** lines of code.

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 
from sympy import true, false

def infer_unknown_values(grid):
  """Fill in any unknown values in the grid that can be inferred.

  Args:
    grid: A list of lists of "F", "U", "S", or "C".

  Returns:
    inferred_grid: A copy of grid with some unknown values replaced.

  Example:
    >> grid = [
    >>   ["F", "U", "C"],
    >>   ["S", "C", "U"],
    >>   ["U", "U", "C"]
    >> ]
    >> infer_unknown_values(grid)
    >> [["F" "S" "C"]
    >>  ["S" "C" "C"]
    >>  ["U" "U" "C"]]
  """
  grid_height = len(grid)
  grid_width = len(grid[0])

  def xy_to_id(x, y):
    return y*grid_width + x

  def id_to_xy(id):
    x = id%grid_width
    y = id//grid_width
    return x, y

  def is_valid(x, y):
    if x < 0 or x >= grid_width:
      return False
    if y < 0 or y >= grid_height:
      return False
    return True

  def get_neighbors(id):
    x, y = id_to_xy(id)
    nei = []

    if is_valid(x-1, y):
      nei.append(xy_to_id(x-1, y))
    if is_valid(x+1, y):
      nei.append(xy_to_id(x+1, y))
    if is_valid(x, y+1):
      nei.append(xy_to_id(x, y+1))
    if is_valid(x, y-1):
      nei.append(xy_to_id(x, y-1))

    return nei

  print("\n\n\n")

  vars = {}
  for y in range(grid_height):
    for x in range(grid_width):
      vars[xy_to_id(x, y)] = {}
      vars[xy_to_id(x, y)]['F'] = symbols(f'{xy_to_id(x, y)}F')
      vars[xy_to_id(x, y)]['S'] = symbols(f'{xy_to_id(x, y)}S')
      vars[xy_to_id(x, y)]['C'] = symbols(f'{xy_to_id(x, y)}C')
      types = ['F', 'S', 'C']
      if grid[y][x] != 'U':
        for t in types:
          if grid[y][x] == t:
            vars[xy_to_id(x, y)][t] = true
          else:
            vars[xy_to_id(x, y)][t] = false

  dummy1, dummy2 = symbols('d1, d2')
  dummy1 = true
  dummy2 = false
  formula = dummy1
  for y in range(grid_height):
    for x in range(grid_width):
      id = xy_to_id(x, y)

      or_nei_fire = dummy2
      and_nei_clear = dummy1
      for nei in get_neighbors(id):
        or_nei_fire = or_nei_fire | vars[nei]['F']
        and_nei_clear = and_nei_clear & vars[nei]['C']

        #FIRE IMPLIES SMOKE
        formula = formula & (~vars[id]['F'] | vars[nei]['S'])

      #SMOKE IMPLIES ONE FIRE
      formula = formula & (~vars[id]['S'] | or_nei_fire)

      #ALL CLEAR IFF CLEAR
      # if grid[y][x] == 'U':
      #   formula = formula & (~and_nei_clear | vars[id]['C'])
      #   formula = formula & (~vars[id]['C'] | and_nei_clear)

      #A THING CAN ONLY HAVE ONE VALUE
      #formula = formula & ((~vars[id]['F'] & ~vars[id]['S'] & vars[id]['C']) | (vars[id]['F'] & ~vars[id]['S'] & ~vars[id]['C']) | (~vars[id]['F'] & vars[id]['S'] & ~vars[id]['C']))

  # dummy1 = True
  # dummy2 = False
  # print(formula)
  # for y in range(grid_height):
  #   for x in range(grid_width):
  #     types = ['F', 'S', 'C']
  #     if grid[y][x] != 'U':
  #       for t in types:
  #         if grid[y][x] == t:
  #           vars[xy_to_id(x, y)][t] = 
  #         else:
  #           vars[xy_to_id(x, y)][t] = False
  print(formula)
  sln = satisfiable(formula)
  print(sln)

  for y in range(grid_height):
    for x in range(grid_width):
      types = ['F', 'S', 'C']
      if grid[y][x] == 'U':
        for t in types:
          sym = vars[xy_to_id(x, y)][t]
          if sym in sln and sln[sym]:
            grid[y][x] = t
            break
        if vars[xy_to_id(x, y)]['F'] in sln and not sln[vars[xy_to_id(x, y)]['F']]:
          if vars[xy_to_id(x, y)]['S'] in sln and not sln[vars[xy_to_id(x, y)]['S']]:
            grid[y][x] = 'C'
  print(grid)
  return grid



  possible_values = {}
  for x in range(height):
    for y in range(width):
      if grid[x][y] != 'U':
        continue
      possible_values[(x, y)] = {}
      types = ['F', 'S', 'C']
      for t in types:
        grid[x][y] = t
        sln = solve(grid)
        if sln != False:
          possible_values[(x, y)][t] = True
      grid[x][y] =  'U'

  print(possible_values)

  for x in range(height):
    for y in range(width):
      if grid[x][y] != 'U':
        continue
      if len(possible_values[(x, y)]) != 1:
        continue
      for v in possible_values[(x, y)]:
        grid[y][x] = v
  

  #FIRE IMPLIES SMOKE
            formula = any_fire & (vars[(x,y)]['F'] >> (vars[(n1,n2)]['S'] & ~vars[(n1,n2)]['F'] & ~vars[(n1,n2)]['C']))

        #SMOKE IMPLIES ONE FIRE
            formula = any_fire & (vars[(x,y)]['S'] >> or_nei_fire)
    sln = satisfiable(formula)
    return sln

  

In [ ]:
from sympy import *
from sympy.logic.inference import satisfiable 
from sympy import true, false

def infer_unknown_values(grid):
  """Fill in any unknown values in the grid that can be inferred.

  Args:
    grid: A list of lists of "F", "U", "S", or "C".

  Returns:
    inferred_grid: A copy of grid with some unknown values replaced.

  Example:
    >> grid = [
    >>   ["F", "U", "C"],
    >>   ["S", "C", "U"],
    >>   ["U", "U", "C"]
    >> ]
    >> infer_unknown_values(grid)
    >> [["F" "S" "C"]
    >>  ["S" "C" "C"]
    >>  ["U" "U" "C"]]
  """
  height = len(grid)
  width = len(grid[0])

  def findNeighbors(m,n, x, y):
    re = []
    xi = ( -1, 1) if 0 < x < m - 1 else ((0, -1) if x > 0 else ((0,) if x == m-1 else (0, 1)))
    yi = (0, -1, 1) if 0 < y < n - 1 else ((0, -1) if y > 0 else ((0,) if y == n-1 else (0, 1)))
    for i in xi:
      if(i != 0):
        re.append((x+i,y))
    for j in yi:
      if(j != 0):
        re.append((x,y+j))
    return re
  

  def solve(grid):
    vars = {}
    for x in range(height):
      for y in range(width):
        vars[(x, y)] = {}
        vars[(x, y)]['F'] = symbols(f'{(x, y)}F')
        vars[(x, y)]['S'] = symbols(f'{(x, y)}S')
        vars[(x, y)]['C'] = symbols(f'{(x, y)}C')
        types = ['F', 'S', 'C']
        if grid[x][y] != 'U':
          for t in types:
            if grid[x][y] == t:
              vars[(x, y)][t] = true
            else:
              vars[(x, y)][t] = false
    print(vars)
    for x in range(height):
        for y in range(width):
          if grid[x][y] == 'U':
            any_fire = symbols("F")
            any_clear = symbols("C")
            all_clear = symbols("AC")
            any_un = symbols("U")
            any_fire = false
            any_clear =false
            all_clear = true
            any_un = false
            for (n1,n2) in findNeighbors(height,width,x,y):
              if grid[n1][n2] != 'U':
                any_fire = any_fire | vars[(n1,n2)]['F']
                any_clear = any_clear | vars[(n1,n2)]['C']
                all_clear = all_clear & vars[(n1,n2)]['C']
              else:
                all_clear = false
                any_un = true
            #some clear no fire implies no fire
            if not any_fire and any_clear:
              vars[(x,y)]['F']= false
            #all clear implies clear
            if all_clear or not any_fire and not any_un:
              vars[(x,y)]['F']= false
              vars[(x,y)]['S']= false
              vars[(x,y)]['C']= true
              grid[x][y] = "C"
              
    changed = True
    while(changed):
      changed = False
      for x in range(height):
        for y in range(width):
          if grid[x][y] == 'U':
              any_fire = false
              any_clear =false
              all_clear = true
              any_possible_fire = false
              for (n1,n2) in findNeighbors(height,width,x,y):
                if grid[n1][n2] != 'U':
                  any_fire = any_fire | vars[(n1,n2)]['F']
                  any_clear = any_clear | vars[(n1,n2)]['C']
                  all_clear = all_clear & vars[(n1,n2)]['C']
                elif not isinstance(vars[(n1,n2)]['F'],tuple):
                  continue
                else:
                  any_possible_fire = true
                  all_clear = false
              if any_fire :
                vars[(x,y)]['C']= false
                vars[(x,y)]['F']= false
                vars[(x,y)]['S']= true
                grid[x][y] = "S"
                changed = True
              if any_clear and not any_fire and not any_possible_fire:
                vars[(x,y)]['F']= false
                vars[(x,y)]['S']= false
                vars[(x,y)]['C']= true
                grid[x][y] = "C"
          if grid[x][y] == 'S':
              any_fire = false
              any_clear =false
              all_clear = true
              any_undefined = false
              for (n1,n2) in findNeighbors(height,width,x,y):
                if grid[n1][n2] != 'U':
                  any_fire = any_fire | vars[(n1,n2)]['F']
                  any_clear = any_clear | vars[(n1,n2)]['C']
                  all_clear = all_clear & vars[(n1,n2)]['C']
                elif not isinstance(vars[(n1,n2)]['F'],tuple):
                  continue
                elif any_undefined:
                  any_undefined = false
                  break
                else:
                  all_clear = false
                  any_undefined = true
                  undefined = (n1,n2)
              if not any_fire and any_undefined:
                vars[undefined]['C']= false
                vars[undefined]['F']= true
                vars[undefined]['S']= false
                grid[undefined[0]][undefined[1]] = "F"
                changed = True
    return grid
           
  print(solve(grid))
  return solve(grid)



    

  

Tests

In [ ]:

assert infer_unknown_values([["U", "F"]]) == [["S", "F"]]

assert infer_unknown_values([["F", "U", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["F", "S", "C"], ["S", "C", "C"], ["U", "U", "C"]]

assert infer_unknown_values([["U", "C", "C"], ["S", "C", "U"], ["U", "U", "C"]]) == [["C", "C", "C"], ["S", "C", "C"], ["F", "S", "C"]]

assert infer_unknown_values([["U", "S", "C", "U"], 
                             ["U", "U", "C", "U"], 
                             ["U", "S", "C", "U"]]) == [["F", "S", "C", "C"], 
                                                        ["S", "C", "C", "C"], 
                                                        ["F", "S", "C", "C"]]

assert infer_unknown_values([["U", "U", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "F", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"]]) == [["C", "C", "C", "U", "U", "U", "U", "U"], ["C", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "U", "U"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "U", "U", "U", "U", "U", "C", "C"], ["U", "C", "U", "S", "U", "U", "U", "U"], ["U", "U", "S", "F", "S", "U", "U", "U"], ["U", "U", "U", "S", "U", "U", "U", "U"]]
print('Tests passed.')

{(0, 0): {'F': ((0, 0)F), 'S': ((0, 0)S), 'C': ((0, 0)C)}, (0, 1): {'F': True, 'S': False, 'C': False}}
[['S', 'F']]
{(0, 0): {'F': False, 'S': True, 'C': False}, (0, 1): {'F': True, 'S': False, 'C': False}}
{(0, 0): {'F': True, 'S': False, 'C': False}, (0, 1): {'F': ((0, 1)F), 'S': ((0, 1)S), 'C': ((0, 1)C)}, (0, 2): {'F': False, 'S': False, 'C': True}, (1, 0): {'F': False, 'S': True, 'C': False}, (1, 1): {'F': False, 'S': False, 'C': True}, (1, 2): {'F': ((1, 2)F), 'S': ((1, 2)S), 'C': ((1, 2)C)}, (2, 0): {'F': ((2, 0)F), 'S': ((2, 0)S), 'C': ((2, 0)C)}, (2, 1): {'F': ((2, 1)F), 'S': ((2, 1)S), 'C': ((2, 1)C)}, (2, 2): {'F': False, 'S': False, 'C': True}}
[['F', 'S', 'C'], ['S', 'C', 'C'], ['U', 'U', 'C']]
{(0, 0): {'F': True, 'S': False, 'C': False}, (0, 1): {'F': False, 'S': True, 'C': False}, (0, 2): {'F': False, 'S': False, 'C': True}, (1, 0): {'F': False, 'S': True, 'C': False}, (1, 1): {'F': False, 'S': False, 'C': True}, (1, 2): {'F': False, 'S': False, 'C': True}, (2, 0): {'F